In [1]:
%matplotlib inline
from matplotlib import pyplot as plt
import numpy as np
import bottleneck as bn

from astropy.nddata import CCDData
from astropy.stats import sigma_clipped_stats
from astropy.coordinates import SkyCoord
from astropy.table import Table, Column, vstack
from astropy.visualization import hist
from astropy import units as u
from astropy.time import Time

from ccdproc import combine, ImageFileCollection

import reproject

from photutils import DAOStarFinder, CircularAperture, CircularAnnulus, aperture_photometry, centroid_sources

from astrowidgets import ImageWidget

## The image viewer widget is optional...

In [2]:
iw = ImageWidget()
iw

ImageWidget(children=(Image(value=b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x00\x00\x01\x00\x01\x00\x00\xff\x…

In [3]:
iw.load_fits('kelt-16-S001-R001-C100-r.fit')
#iw.load_fits('kelt-16-b-S001-R001-C130-r.fit')

the RADECSYS keyword is deprecated, use RADESYSa. [astropy.wcs.wcs]


In [4]:
iw.center_on(SkyCoord(313.92459686592696, 31.78329193408578, unit='degree'), pixel_coords_offset=0)

In [5]:
iw.center_on(SkyCoord.from_name('kelt-16'))

# Required stuff starts here

## Read one image to use detect sources

This could be any image; I chose one roughly in the middle of the sequence. Ideally maybe we'd choose sources that were only in all of the images, but this will work for now.

In [6]:
ccd = CCDData.read('kelt-16-S001-R001-C100-r.fit')

the RADECSYS keyword is deprecated, use RADESYSa. [astropy.wcs.wcs]


In [7]:
def faster_sigma_clip_stats(data, sigma=5, iters=5, axis=None):
    data = data.copy()
    for _ in range(iters):
        central = bn.nanmedian(data, axis=axis)
        try:
            central = central[:, np.newaxis]
        except (ValueError, IndexError):
            pass
                
        std_dif = 1.4826 * bn.nanmedian(np.abs(data - central))
        clips = np.abs(data - central) / std_dif > sigma
        if clips.sum() == 0:
            break
        data[clips] = np.nan
    return bn.nanmean(data, axis=axis), bn.nanmedian(data, axis=axis), bn.nanstd(data, axis=axis)

The background light from the sky needs to be removed before looking for sources. We'll do that by calculating the median. "Sigma clipping" is the term for excluding data that is far from the average...here we do it mostly to exclude the "bright" pixels (i.e. the stars) from our estimate of the background.

In [8]:
men, med, std = faster_sigma_clip_stats(ccd.data, sigma=5)
men, med, std

C:\Anaconda3\envs\ast366\lib\site-packages\ipykernel_launcher.py:11: RuntimeWarning: invalid value encountered in greater
  # This is added back by InteractiveShellApp.init_path()
C:\Anaconda3\envs\ast366\lib\site-packages\ipykernel_launcher.py:11: RuntimeWarning: invalid value encountered in greater
  # This is added back by InteractiveShellApp.init_path()
C:\Anaconda3\envs\ast366\lib\site-packages\ipykernel_launcher.py:11: RuntimeWarning: invalid value encountered in greater
  # This is added back by InteractiveShellApp.init_path()


(21.55644, 21.191854, 8.735692)

This sets up the source detection. The FWHM turns out to be key...making it too small results in a single star being detected as two separate sources.

Stars must be brighter than the threshold to count as sources. Making the number higher gives you fewer detected sources, lower gives you more. There is no "magic" number.

In [9]:
dao = DAOStarFinder(threshold=10 * std, fwhm=8, exclude_border=True)

Actually detect the stars...

In [10]:
stars = dao(ccd - med)

## Only include this if you have an image viewer above

It is handy to see where the detected sources are....

In [11]:
iw.reset_markers()
iw.marker = {'type': 'circle', 'color': 'lightgreen', 'radius': 10}
iw.add_markers(stars, x_colname='xcentroid', y_colname='ycentroid', pixel_coords_offset=0)

## Pick aperture and annulus sizes

Probably the best way to do this for now is to upload one of the kelt-16 images to the notebook we used in class to plot star profiles. You want an aperture about 2×FWHM, an annulus with  an inner radius at least 10 pixels larger than that, and outer radius about 15 pixels larger than the inner.

In [12]:
aperture_rad = 12
inner_annulus = aperture_rad + 15
outer_annulus = inner_annulus + 15

Set up the aperture objects used later by photutils to do the photometry.

In [13]:
aps = CircularAperture([stars['xcentroid'], stars['ycentroid']], r=aperture_rad)
anuls = CircularAnnulus([stars['xcentroid'], stars['ycentroid']], inner_annulus, outer_annulus)

## Set pixels that are above point where CCD becomes non-linear to invalid value

This will ensure that the aperture sum is an invalid value if one or more of the pixels in the aperture is non-linear. *Change the value below to what you think is appropriate -- I'm going from memory.*

In [14]:
max_adu = 45000
ccd.data[ccd.data > max_adu] = np.nan

## Set up sky coordinates of target list

We'll want to the list of stars in RA/Dec so we can find them in each image. 

`star_locs` is waht is used later on to place the apertures

In [15]:
star_locs = ccd.wcs.all_pix2world(stars['xcentroid'], stars['ycentroid'], 0)

### Check for stars closer than 2 × aperture radius and remove from list

In [16]:
star_coords = SkyCoord(ra=star_locs[0], dec=star_locs[1], frame='icrs', unit='degree')

In [17]:
idxc, d2d, d3d = star_coords.match_to_catalog_sky(star_coords, nthneighbor=2)

In [18]:
d2d.min()

<Angle 0.00082005 deg>

In [19]:
too_close = d2d < (aperture_rad * 2 * 0.563 * u.arcsec)

### Include this only if you have the image viewer -- it shows which stars are being removed

In [20]:
iw.marker = {'type': 'circle', 'color': 'red', 'radius': 20}
iw.add_markers(stars[too_close], x_colname='xcentroid', y_colname='ycentroid', pixel_coords_offset=0)

In [21]:
star_locs = (star_locs[0][~too_close], star_locs[1][~too_close])

### Make up a unique ID for each star

In [22]:
star_ids = np.arange(len(star_locs[0])) + 1

## Define a couple of convenience functions

### Calculate average pixel values in annulus, rejecting extreme values

In [23]:
def clipped_sky_per_pix_stats(data, annulus, sigma=5, iters=5):
    # Get a list of masks from the annuli
    # Use the 'center' method because then pixels are either in or out. To use
    # 'partial' or 'exact' we would need to do a weighted sigma clip and I'm not sure 
    # how to do that.
    masks = annulus.to_mask(method='center')
    
    anul_list = []
    for mask in masks:
        # Multiply the mask times the data
        to_clip = mask.multiply(data.data, fill_value=np.nan)
        anul_list.append(to_clip.flatten())
    # Convert the list to an array for doing the sigma clip
    anul_array = np.array(anul_list)
    # Turn all zeros into np.nan...
    anul_array[anul_array == 0] = np.nan
    avg_sky_per_pix, med_sky_per_pix, std_sky_per_pix = faster_sigma_clip_stats(anul_array, 
                                                                                sigma=sigma, 
                                                                                iters=iters,
                                                                                axis=1
                                                                               )

    return (avg_sky_per_pix * data.unit, med_sky_per_pix * data.unit, std_sky_per_pix * data.unit)

### Add more columns to the data table 

In [24]:
def add_to_photometry_table(phot, ccd, annulus, apertures, fname='', 
                            star_ids=None, gain=None):
    phot.rename_column('aperture_sum_0', 'aperture_sum')
    phot['aperture_sum'].unit = u.adu
    phot.rename_column('aperture_sum_1', 'annulus_sum')
    star_locs = ccd.wcs.all_pix2world(phot['xcenter'], phot['ycenter'], 0)
    star_coords = SkyCoord(ra=star_locs[0], dec=star_locs[1], frame='icrs', unit='degree')
    phot['RA'] = star_coords.ra
    phot['Dec'] = star_coords.dec
    print('        ...calculating clipped sky stats')
    avg_sky_per_pix, med_sky_per_pix, std_sky_per_pix = clipped_sky_per_pix_stats(ccd, annulus)
    print('        ...DONE calculating clipp sky stats')
    phot['sky_per_pix_avg'] = avg_sky_per_pix
    phot['sky_per_pix_med'] = med_sky_per_pix
    phot['sky_per_pix_std'] = std_sky_per_pix
    phot['aperture'] = apertures.r * u.pixel
    phot['aperture_area'] = apertures.area() # * u.pixel * u.pixel
    phot['annulus_inner'] = annulus.r_in * u.pixel
    phot['annulus_outer'] = annulus.r_out * u.pixel
    phot['annulus_area'] = annulus.area() #* u.pixel * u.pixel
    phot['exposure'] = [ccd.header['exposure']] * len(phot) * u.second
    phot['date-obs'] = [ccd.header['DATE-OBS']] * len(phot)
    night = Time(ccd.header['DATE-OBS'], scale='utc')
    night.format = 'mjd'
    phot['night'] = np.int(np.floor(night.value - 0.5))
    phot['aperture_net_flux'] = phot['aperture_sum'] - phot['aperture_area'] * phot['sky_per_pix_avg']
    
    if gain is not None:
        phot['mag_inst_{}'.format(ccd.header['filter'])] = -2.5 * np.log10(gain * phot['aperture_net_flux'].value / phot['exposure'].value)

    metadata_to_add = ['AIRMASS', 'FILTER']
    for meta in metadata_to_add:
        phot[meta.lower()] = [ccd.header[meta]] * len(phot)    
    if fname:
        phot['file'] = fname
    if star_ids is not None:
        phot['star_id'] = star_ids

## We are ready to do photometry on all of the images

### Get the images in this folder...

In [25]:
ifc = ImageFileCollection('.')

### Loop over all images and do photometry on them. 

+ Change the object name if you need to.
+ If there are images you want to skip put them in a different folder

In [26]:
phots = []
missing_stars = []
for a_ccd, fname in ifc.ccds(object='kelt-16b', return_fname=True):
    print('on image ', fname)
    # Convert RA/Dec to pixel coordinates for this image
    pix_coords = a_ccd.wcs.all_world2pix(star_locs[0], star_locs[1], 0)
    xs, ys = pix_coords
    
    # Remove anything that is too close to the edges/out of frame
    padding = 3 * aperture_rad
    out_of_bounds = (xs < padding) | (xs > (a_ccd.shape[0] - padding)) | (ys < padding) | (ys > (a_ccd.shape[1] - padding))
    in_bounds = ~out_of_bounds
    
    # Find centroids of each region around star that is in_bounds
    xs_in = xs[in_bounds]
    ys_in = ys[in_bounds]
    print('    ...finding centroids')
    xcen, ycen = centroid_sources(a_ccd.data, xs_in, ys_in, box_size=2 * aperture_rad + 1)

    # Calculate offset between centroid in this image and the positions based on 
    # input RA/Dec. Later we wil set the magnitude of those with large differences
    # to an invalid value (maybe).
    center_diff = np.sqrt((xs_in - xcen)**2 + (ys_in - ycen)**2)

    #Set up apertures and annuli based on the centroids in this image.
    aps = CircularAperture((xcen, ycen), r=aperture_rad)
    anuls = CircularAnnulus((xcen, ycen), inner_annulus, outer_annulus)

    # Set any clearly bad values to NaN
    a_ccd.data[a_ccd.data > max_adu] = np.nan
    print('    ...doing photometry')
    # Do the photometry...
    pho = aperture_photometry(a_ccd.data, (aps, anuls), mask=a_ccd.mask, method='center')

    # We may have some stars we did not do photometry for because those stars were out of bounds.
    # Add the ones we missed to the list of missing
    missed = star_ids[out_of_bounds]
    missing_stars.append(missed)

    # Add all the extra goodies to the table
    print('    ...adding extra columns')
    add_to_photometry_table(pho, a_ccd, anuls, aps, fname=fname, star_ids=star_ids[in_bounds],
                            gain=1.47)
    # And add the final table to the list of tables
    phots.append(pho)

the RADECSYS keyword is deprecated, use RADESYSa. [astropy.wcs.wcs]
the RADECSYS keyword is deprecated, use RADESYSa.


on image  kelt-16-S001-R001-C001-r.fit
    ...finding centroids
    ...doing photometry
    ...adding extra columns
        ...calculating clipped sky stats


C:\Anaconda3\envs\ast366\lib\site-packages\ipykernel_launcher.py:11: RuntimeWarning: invalid value encountered in greater
  # This is added back by InteractiveShellApp.init_path()


        ...DONE calculating clipp sky stats
on image  kelt-16-S001-R001-C002-r.fit
    ...finding centroids


C:\Anaconda3\envs\ast366\lib\site-packages\ipykernel_launcher.py:29: RuntimeWarning: invalid value encountered in log10


    ...doing photometry
    ...adding extra columns
        ...calculating clipped sky stats
        ...DONE calculating clipp sky stats
on image  kelt-16-S001-R001-C003-r.fit
    ...finding centroids
    ...doing photometry
    ...adding extra columns
        ...calculating clipped sky stats
        ...DONE calculating clipp sky stats
on image  kelt-16-S001-R001-C004-r.fit
    ...finding centroids
    ...doing photometry
    ...adding extra columns
        ...calculating clipped sky stats
        ...DONE calculating clipp sky stats
on image  kelt-16-S001-R001-C005-r.fit
    ...finding centroids
    ...doing photometry
    ...adding extra columns
        ...calculating clipped sky stats
        ...DONE calculating clipp sky stats
on image  kelt-16-S001-R001-C006-r.fit
    ...finding centroids
    ...doing photometry
    ...adding extra columns
        ...calculating clipped sky stats
        ...DONE calculating clipp sky stats
on image  kelt-16-S001-R001-C007-r.fit
    ...finding centr

    ...doing photometry
    ...adding extra columns
        ...calculating clipped sky stats
        ...DONE calculating clipp sky stats
on image  kelt-16-S001-R001-C044-r.fit
    ...finding centroids
    ...doing photometry
    ...adding extra columns
        ...calculating clipped sky stats
        ...DONE calculating clipp sky stats
on image  kelt-16-S001-R001-C045-r.fit
    ...finding centroids
    ...doing photometry
    ...adding extra columns
        ...calculating clipped sky stats
        ...DONE calculating clipp sky stats
on image  kelt-16-S001-R001-C046-r.fit
    ...finding centroids
    ...doing photometry
    ...adding extra columns
        ...calculating clipped sky stats
        ...DONE calculating clipp sky stats
on image  kelt-16-S001-R001-C047-r.fit
    ...finding centroids
    ...doing photometry
    ...adding extra columns
        ...calculating clipped sky stats
        ...DONE calculating clipp sky stats
on image  kelt-16-S001-R001-C048-r.fit
    ...finding centr

    ...finding centroids
    ...doing photometry
    ...adding extra columns
        ...calculating clipped sky stats
        ...DONE calculating clipp sky stats
on image  kelt-16-S001-R001-C085-r.fit
    ...finding centroids
    ...doing photometry
    ...adding extra columns
        ...calculating clipped sky stats
        ...DONE calculating clipp sky stats
on image  kelt-16-S001-R001-C086-r.fit
    ...finding centroids
    ...doing photometry
    ...adding extra columns
        ...calculating clipped sky stats
        ...DONE calculating clipp sky stats
on image  kelt-16-S001-R001-C087-r.fit
    ...finding centroids
    ...doing photometry
    ...adding extra columns
        ...calculating clipped sky stats
        ...DONE calculating clipp sky stats
on image  kelt-16-S001-R001-C088-r.fit
    ...finding centroids
    ...doing photometry
    ...adding extra columns
        ...calculating clipped sky stats
        ...DONE calculating clipp sky stats
on image  kelt-16-S001-R001-C089-r

    ...doing photometry
    ...adding extra columns
        ...calculating clipped sky stats
        ...DONE calculating clipp sky stats
on image  kelt-16-S001-R001-C126-r.fit
    ...finding centroids
    ...doing photometry
    ...adding extra columns
        ...calculating clipped sky stats
        ...DONE calculating clipp sky stats
on image  kelt-16-S001-R001-C127-r.fit
    ...finding centroids
    ...doing photometry
    ...adding extra columns
        ...calculating clipped sky stats
        ...DONE calculating clipp sky stats
on image  kelt-16-S001-R001-C128-r.fit
    ...finding centroids
    ...doing photometry
    ...adding extra columns
        ...calculating clipped sky stats
        ...DONE calculating clipp sky stats
on image  kelt-16-S001-R001-C129-r.fit
    ...finding centroids
    ...doing photometry
    ...adding extra columns
        ...calculating clipped sky stats
        ...DONE calculating clipp sky stats
on image  kelt-16-S001-R001-C130-r.fit
    ...finding centr

    ...doing photometry
    ...adding extra columns
        ...calculating clipped sky stats
        ...DONE calculating clipp sky stats


### Combine all of the individual photometry tables into one

In [27]:
all_phot = vstack(phots)

### Eliminate any stars that are missing from one or more images

This makes life a little easier later...

In [28]:
uniques = set()
for miss in missing_stars:
    uniques.update(set(miss))

actually_bad = sorted([u for u in uniques if u in all_phot['star_id']])
len(uniques), len(actually_bad)

(147, 144)

In [29]:
all_phot.add_index('star_id')
bad_rows = all_phot.loc_indices[actually_bad]

In [30]:
all_phot.remove_indices('star_id')
all_phot.remove_rows(sorted(bad_rows))

### Write out the file!

In [31]:
all_phot

id,xcenter,ycenter,aperture_sum,annulus_sum,RA,Dec,sky_per_pix_avg,sky_per_pix_med,sky_per_pix_std,aperture,aperture_area,annulus_inner,annulus_outer,annulus_area,exposure,date-obs,night,aperture_net_flux,mag_inst_r,airmass,filter,file,star_id
,pix,pix,adu,,deg,deg,adu,adu,adu,pix,,pix,pix,,s,,,adu,,,,,
int32,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,str19,int32,float64,float64,float64,str1,str28,int32
14,3154.0652623290325,71.08640382072655,19636.398203849792,101218.21279919147,314.5346064763514,31.395607446904748,31.061781630824637,30.862295150756836,9.440040360769093,12.0,452.3893421169302,27.0,42.0,3251.548396465436,60.0,2018-07-24T05:19:53,58322,5584.379246901288,-5.340352472074655,1.121,r,kelt-16-S001-R001-C001-r.fit,25
15,2554.7526840417163,73.12639199314577,19241.350580215454,104729.48829507828,314.4270501981604,31.376843537644437,31.635215246112548,31.140750885009766,9.390828652308281,12.0,452.3893421169302,27.0,42.0,3251.548396465436,60.0,2018-07-24T05:19:53,58322,4929.916367299118,-5.205014090479365,1.121,r,kelt-16-S001-R001-C001-r.fit,26
16,2839.0427405315436,73.11079450374831,16872.513807296753,101179.7171254158,314.47803061074865,31.385900921348142,30.75727853677831,30.633115768432617,9.196566862099427,12.0,452.3893421169302,27.0,42.0,3251.548396465436,60.0,2018-07-24T05:19:53,58322,2958.248804736437,-4.650501955195355,1.121,r,kelt-16-S001-R001-C001-r.fit,27
17,2920.9222395184966,74.06026813848437,19898.36015367508,101704.91505388916,314.4926796799292,31.388652037915236,31.259383215270375,30.804100036621094,9.339615905490392,12.0,452.3893421169302,27.0,42.0,3251.548396465436,60.0,2018-07-24T05:19:53,58322,5756.948345937904,-5.37339604285437,1.121,r,kelt-16-S001-R001-C001-r.fit,28
18,2484.946540721911,79.09943889447156,19583.046536445618,177530.46922445297,314.41431035010214,31.375530380708284,34.88524456857478,33.44371032714844,11.477046172063483,12.0,452.3893421169302,27.0,42.0,3251.548396465436,60.0,2018-07-24T05:19:53,58322,3801.333696479862,-4.922755199427578,1.121,r,kelt-16-S001-R001-C001-r.fit,29
19,1088.0836400127873,85.0256783521898,16784.450025081635,108935.89793747663,314.1637277785051,31.33161702727585,31.672472889487587,31.206933975219727,9.686621599261443,12.0,452.3893421169302,27.0,42.0,3251.548396465436,60.0,2018-07-24T05:19:53,58322,2456.1608513900374,-4.448557223134261,1.121,r,kelt-16-S001-R001-C001-r.fit,30
20,3906.074337014607,84.39173162591916,35575.36714363098,105543.29313248396,314.66903038622775,31.421459732181386,32.123997812849524,31.747995376586914,10.224443519062163,12.0,452.3893421169302,27.0,42.0,3251.548396465436,60.0,2018-07-24T05:19:53,58322,21042.81290691028,-6.780674695563143,1.121,r,kelt-16-S001-R001-C001-r.fit,31
21,2070.926989829942,89.05244260647149,21344.527408599854,124453.23524141312,314.3397111957455,31.363818141684096,35.85087647237197,35.455596923828125,9.557938670145557,12.0,452.3893421169302,27.0,42.0,3251.548396465436,60.0,2018-07-24T05:19:53,58322,5125.972986948165,-5.247355993892086,1.121,r,kelt-16-S001-R001-C001-r.fit,32


In [33]:
all_phot.write('all_the_photometry.fits')

In [34]:
all_phot.write('all_the_photometry.csv')